In [ ]:
%%bash
cd /home/workspace/2022.micro.artifact/evaluation_setups/unstructured_eval_updated/scripts
echo "Running evaluation sweep for unstructured sparse architecture"
time python sweep.py

In [1]:
%%bash
cd /home/workspace/2022.micro.artifact/evaluation_setups/unstructured_eval_updated/scripts
time python parse_and_plot1.py

Parsing results across all workloads and density pairs...

--- Processing workload: resnet ---

Summary Statistics (Resnet Workload):
  Density (A=0.3, B=0.4):
    Cycles: 1,627,810
    Energy: 1,566,460,000.0 pJ
    Utilization: 27.00%
  Density (A=0.3, B=1.0):
    Cycles: 2,909,566
    Energy: 2,771,080,000.0 pJ
    Utilization: 37.00%
  Density (A=0.5, B=0.4):
    Cycles: 2,421,382
    Energy: 2,257,960,000.0 pJ
    Utilization: 30.00%
  Density (A=0.5, B=1.0):
    Cycles: 4,328,007
    Energy: 3,801,320,000.0 pJ
    Utilization: 42.00%
  Density (A=0.7, B=0.4):
    Cycles: 3,213,093
    Energy: 2,948,260,000.0 pJ
    Utilization: 31.00%
  Density (A=0.7, B=1.0):
    Cycles: 5,743,120
    Energy: 4,830,370,000.0 pJ
    Utilization: 44.00%
  Density (A=0.9, B=0.4):
    Cycles: 4,029,472
    Energy: 3,638,520,000.0 pJ
    Utilization: 32.00%
  Density (A=0.9, B=1.0):
    Cycles: 7,202,327
    Energy: 5,859,470,000.0 pJ
    Utilization: 45.00%
  Density (A=1.0, B=0.4):
    Cycles: 4,04


real	0m1.822s
user	0m3.542s
sys	0m4.543s


In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import os

# Define path to the figure using the absolute path
figures_base_dir = "../evaluation_setups/unstructured_eval_updated/outputs/figures/"
resnet_unstructured_path =  os.path.join(figures_base_dir,"resnet_sparsity_comparison.png")
mobilenet_unstructured_path = os.path.join(figures_base_dir, "mobilenet_sparsity_comparison.png")
alexnet_unstructured_path = os.path.join(figures_base_dir, "alexnet_sparsity_comparison.png")

#displaying the figures
if os.path.exists(resnet_unstructured_path):
    print("Displaying ResNet Comparison:")
    display(Image(filename=resnet_unstructured_path))
else:
    print(f"ResNet plot not found at: {resnet_unstructured_path}")

if os.path.exists(mobilenet_unstructured_path):
    print("\nDisplaying MobileNet Comparison:")
    display(Image(filename=mobilenet_unstructured_path))
else:
    print(f"MobileNet plot not found at: {mobilenet_unstructured_path}")

if os.path.exists(alexnet_unstructured_path):
    print("\nDisplaying AlexNet Comparison:")
    display(Image(filename=alexnet_unstructured_path))
else:
    print(f"AlexNet plot not found at: {alexnet_unstructured_path}")

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import re
import matplotlib.pyplot as plt

# Function to parse stats directly from the text file
def parse_stats_file(stats_file):
    if not os.path.exists(stats_file):
        print(f"Warning: Stats file not found: {stats_file}")
        return None
    
    stats = {'level_energy': {}}
    try:
        with open(stats_file, 'r') as f:
            content = f.read()
            
            # --- Fix 1: Extract total energy from Summary Stats --- 
            # Look for "Energy: VALUE uJ" in the Summary Stats section
            summary_energy_match = re.search(r'Summary Stats\n-{3,}\n.*?Energy:\s+([\d.]+)\s+uJ', content, re.DOTALL)
            if summary_energy_match:
                stats['energy'] = float(summary_energy_match.group(1)) * 1_000_000 # Convert uJ to pJ
            else:
                # Fallback: Try finding any "Energy (total)" line if summary is missing (less reliable)
                energy_match = re.search(r'Energy \(total\)\s+:\s+([\d.]+)', content)
                if energy_match:
                    print(f"Warning: Using fallback total energy parsing for {stats_file}")
                    stats['energy'] = float(energy_match.group(1))
                else:
                    print(f"Error: Could not parse total energy from {stats_file}")
                    stats['energy'] = 0 # Assign 0 if unparsable

            # Extract cycles (Prefer summary stats if available)
            summary_cycles_match = re.search(r'Summary Stats\n-{3,}\n.*?Cycles:\s+(\d+)', content, re.DOTALL)
            if summary_cycles_match:
                stats['cycles'] = int(summary_cycles_match.group(1))
            else:
                # Fallback to first Cycles match
                cycles_match = re.search(r'Cycles\s+:\s+(\d+)', content)
                if cycles_match:
                     print(f"Warning: Using fallback cycles parsing for {stats_file}")
                     stats['cycles'] = int(cycles_match.group(1))
                else:
                     stats['cycles'] = 0


            # Extract utilization (Prefer summary stats: 0.0-1.0 scale)
            summary_util_match = re.search(r'Summary Stats\n-{3,}\n.*?Utilization:\s+([\d.]+)', content, re.DOTALL)
            if summary_util_match:
                # Convert utilization (0.0-1.0) to percentage for display consistency
                stats['utilization'] = float(summary_util_match.group(1)) 
            else:
                # Fallback: MAC level utilization (Average instances)
                # This requires knowing total instances to be meaningful %
                util_match = re.search(r'Level 0\n-{3,}\n=== MAC ===.*?Utilized instances \(average\)\s+:\s+([\d.]+)', content, re.DOTALL)
                if util_match:
                     print(f"Warning: Using fallback MAC utilization for {stats_file}. Value is avg instances, not percentage.")
                     # Store the average instance count, not a percentage
                     stats['utilization'] = float(util_match.group(1)) 
                else:
                    stats['utilization'] = 0
                
            # --- Fix 2: Extract per-level energy, summing across tensors --- 
            # Regex to capture content between "Level X..." and the next "Level Y..." or end of file
            level_pattern = r'(Level\s+(\d+)\n-{3,}\n=== ([^\n]+) ===.*?)(?=(Level\s+\d+\n-{3,})|\Z)'
            level_matches = re.finditer(level_pattern, content, re.DOTALL)
            
            level_energy = {}
            parsed_sum = 0.0
            for match in level_matches:
                level_full_header = match.group(1).strip() # Includes "Level X..." etc.
                level_num = int(match.group(2))
                level_name = match.group(3).strip()
                # Content is the full block captured by group 1
                level_content = match.group(1) 
                
                # Find *all* "Energy (total) : VALUE pJ" lines within this level's block
                energy_values = re.findall(r'Energy \(total\)\s+:\s+([\d.]+) pJ', level_content)
                
                # Sum up all found energy values for this level
                total_level_energy = sum(float(e.replace(',', '')) for e in energy_values) # Handle potential commas
                
                level_key = f"L{level_num}_{level_name}"
                if total_level_energy > 0:
                     level_energy[level_key] = total_level_energy
                     parsed_sum += total_level_energy
                # If a level exists but has 0 energy reported, we might want to record that
                elif level_full_header: # Check if the level header was actually found
                     level_energy[level_key] = 0.0

            stats['level_energy'] = level_energy
                
        # Add a simple validation check
        if stats['energy'] > 0 and not np.isclose(parsed_sum, stats['energy'], rtol=0.1): # Allow 10% tolerance for potential rounding
             print(f"Warning: Sum of parsed level energies ({parsed_sum:,.2f} pJ) doesn't closely match total summary energy ({stats['energy']:,.2f} pJ) for {stats_file}. Check parsing logic or stats file format.")
        elif stats['energy'] == 0 and parsed_sum > 0:
             print(f"Warning: Total summary energy is 0, but parsed level energy sum is {parsed_sum:,.2f} pJ for {stats_file}.")


        return stats
    except Exception as e:
        print(f"Critical Error parsing {stats_file}: {e}")
        # Consider logging the error traceback here
        return None

# Load and process the results for both architectures
workloads = ["resnet50_conv1", "alexnet_conv1_sparse", "mobilenet_conv1_sparse"]
workload_display_names = {'resnet50_conv1': 'ResNet-50', 
                          'alexnet_conv1_sparse': 'AlexNet', 
                          'mobilenet_conv1_sparse': 'MobileNet'}

unstructured_results = {}
ideal_results = {}

for workload in workloads:
    # Unstructured sparse architecture results
    unstructured_stats_file = f"../evaluation_setups/unstructured_sparse_eval/outputs/unstructured_{workload}/timeloop-mapper.stats.txt"
    stats = parse_stats_file(unstructured_stats_file)
    if stats:
        unstructured_results[workload] = stats
    
    # Ideal sparse architecture results
    ideal_stats_file = f"../evaluation_setups/unstructured_sparse_eval/outputs/ideal_{workload}/timeloop-mapper.stats.txt"
    stats = parse_stats_file(ideal_stats_file)
    if stats:
        ideal_results[workload] = stats

# Create DataFrames for comparison
def create_dataframe(results, workloads):
    data = []
    for workload in workloads:
        if workload in results:
            row = {
                'Workload': workload_display_names[workload],
                'Cycles': results[workload].get('cycles', 0),
                'Energy (pJ)': results[workload].get('energy', 0),
                'Utilization (%)': results[workload].get('utilization', 0) * 100  # Convert to percentage
            }
            data.append(row)
    return pd.DataFrame(data)

unstructured_df = create_dataframe(unstructured_results, workloads)
ideal_df = create_dataframe(ideal_results, workloads)

# Calculate improvement ratios
if not unstructured_df.empty and not ideal_df.empty:
    # Make sure both dataframes have the same workloads
    common_workloads = set(unstructured_df['Workload']).intersection(set(ideal_df['Workload']))
    unstructured_filtered = unstructured_df[unstructured_df['Workload'].isin(common_workloads)]
    ideal_filtered = ideal_df[ideal_df['Workload'].isin(common_workloads)]
    
    # Sort both DataFrames by workload to ensure they match
    unstructured_filtered = unstructured_filtered.sort_values('Workload').reset_index(drop=True)
    ideal_filtered = ideal_filtered.sort_values('Workload').reset_index(drop=True)
    
    comparison_df = pd.DataFrame({
        'Workload': unstructured_filtered['Workload'],
        'Cycle Ratio (Unstructured/Ideal)': unstructured_filtered['Cycles'] / ideal_filtered['Cycles'].replace(0, np.nan),
        'Energy Ratio (Unstructured/Ideal)': unstructured_filtered['Energy (pJ)'] / ideal_filtered['Energy (pJ)'].replace(0, np.nan),
        'Utilization Ratio (Ideal/Unstructured)': ideal_filtered['Utilization (%)'] / unstructured_filtered['Utilization (%)'].replace(0, np.nan)
    })
else:
    comparison_df = pd.DataFrame(columns=['Workload', 'Cycle Ratio (Unstructured/Ideal)', 
                                          'Energy Ratio (Unstructured/Ideal)', 'Utilization Ratio (Ideal/Unstructured)'])

# Energy breakdown by hardware level
print("Energy Breakdown by Hardware Level:")
print("=" * 60)

for arch_name, results in [('Unstructured', unstructured_results), ('Ideal', ideal_results)]:
    print(f"\n{arch_name} Architecture:")
    
    for workload in workloads:
        if workload in results and 'level_energy' in results[workload]:
            level_energy = results[workload]['level_energy']
            total_energy = results[workload].get('energy', 0)
            
            if level_energy and total_energy > 0:
                print(f"\n{workload_display_names[workload]}:")
                
                for level, energy in level_energy.items():
                    percentage = (energy / total_energy) * 100
                    print(f"  {level}: {energy:,.2f} pJ ({percentage:.2f}%)")
            else:
                print(f"\n{workload_display_names[workload]}: Energy breakdown not available")

# Display the summary analysis
print("\nPerformance Comparison: Unstructured vs Ideal")
print("=" * 60)
print("Unstructured Sparse Architecture:")
if not unstructured_df.empty:
    print(unstructured_df.to_string(index=False))
else:
    print("No data available")

print("\nIdeal Sparse Architecture:")
if not ideal_df.empty:
    print(ideal_df.to_string(index=False))
else:
    print("No data available")

print("\nImprovement Ratios:")
if not comparison_df.empty:
    print(comparison_df.to_string(index=False))
else:
    print("No data available for comparison")

print("\nKey Observations:")
print("-" * 60)

if not comparison_df.empty:
    # Filter out NaN and infinity values
    cycle_ratios = comparison_df['Cycle Ratio (Unstructured/Ideal)'].replace([np.inf, -np.inf], np.nan).dropna()
    energy_ratios = comparison_df['Energy Ratio (Unstructured/Ideal)'].replace([np.inf, -np.inf], np.nan).dropna()
    
    if not cycle_ratios.empty:
        avg_cycle_ratio = cycle_ratios.mean()
        print(f"1. Average cycle overhead from sparse metadata: {avg_cycle_ratio:.2f}x")
    else:
        print("1. Cannot calculate average cycle overhead (insufficient data)")
    
    if not energy_ratios.empty:
        avg_energy_ratio = energy_ratios.mean()
        print(f"2. Average energy overhead from sparse metadata: {avg_energy_ratio:.2f}x")
    else:
        print("2. Cannot calculate average energy overhead (insufficient data)")
    
    print("\n3. Workload-Specific Impact:")
    for _, row in comparison_df.iterrows():
        cycle_ratio = row['Cycle Ratio (Unstructured/Ideal)']
        energy_ratio = row['Energy Ratio (Unstructured/Ideal)']
        
        result_str = f"   - {row['Workload']}: "
        if not pd.isna(cycle_ratio) and not np.isinf(cycle_ratio):
            result_str += f"{cycle_ratio:.2f}x cycle overhead, "
        else:
            result_str += "cycle data unavailable, "
            
        if not pd.isna(energy_ratio) and not np.isinf(energy_ratio):
            result_str += f"{energy_ratio:.2f}x energy overhead"
        else:
            result_str += "energy data unavailable"
            
        print(result_str)
else:
    print("Not enough data available for meaningful observations")

print("\n4. Recommendations for Hardware Optimization:")
print("   - Optimize hardware for sparse metadata handling to reduce cycle and energy overhead")
print("   - Focus optimizations on memory levels with highest energy consumption")
print("   - Consider workload-specific accelerator configurations based on observed patterns")